In [5]:
import pandas as pd

In [51]:
df = pd.read_csv('../../../dades/raposos/certificats-ine-raposos.csv', dtype={'referencia_cadastral':str, 'codi_districte':str, 'codi_postal':str})



/var/folders/0b/9qg1xh5d46j8phx5051jbncw0000gn/T/ipykernel_78823/932704025.py:1: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../../dades/raposos/certificats-ine-raposos.csv', dtype={'referencia_cadastral':str, 'codi_districte':str, 'codi_postal':str})


In [52]:
# Transformar data_entrada
df_processed = df
df_processed['data_entrada'] = pd.to_datetime(df['data_entrada'])

# Filtrar 'us_habitatge' distinto de 'Terciari'
df_processed = df_processed[df_processed['us_edifici'] != 'Terciari']

In [53]:
res = {}

idx_energetics = {
    "A": 1,
    "B": 2,
    "C": 3,
    "D": 4,
    "E": 5,
    "F": 6,
    "G": 7,
}

for index, row in df_processed.iterrows():
    if not row['referencia_cadastral'] in res:
        res[row['referencia_cadastral']] = {
            "referencia_cadastral": row['referencia_cadastral'],
            "codi_districte": row['codi_districte'],
            'codi_postal': row['codi_postal'],
            'poblacio': row['poblacio'],
            'comarca': row['comarca'],
            'nom_provincia': row['nom_provincia'],
            'codi_poblacio': row['codi_poblacio'],
            'codi_comarca': row['codi_comarca'],
            'codi_provincia': row['codi_provincia'],
            "year": None,
            "qualificacio_consum_energia_no_renovables": None,
            "renta_neta_mitja_por_persona": None,
            "indice_gini": None,
        }
    
    if res[row['referencia_cadastral']]['year'] is None or res[row['referencia_cadastral']]['year'] < row['data_entrada'].year:
        res[row['referencia_cadastral']]['year'] = row['data_entrada'].year
        res[row['referencia_cadastral']]['qualificacio_consum_energia_no_renovables'] = idx_energetics[row['qualificacio_consum_energia_no_renovables']]
        res[row['referencia_cadastral']]['renta_neta_mitja_por_persona'] = row['renta_neta_mitja_por_persona']
        res[row['referencia_cadastral']]['indice_gini'] = row['indice_gini']


In [54]:
df_main = pd.DataFrame(res.values())

In [55]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1078882 entries, 0 to 1078881
Data columns (total 13 columns):
 #   Column                                     Non-Null Count    Dtype  
---  ------                                     --------------    -----  
 0   referencia_cadastral                       1078881 non-null  object 
 1   codi_districte                             838737 non-null   object 
 2   codi_postal                                1078882 non-null  object 
 3   poblacio                                   1078882 non-null  object 
 4   comarca                                    1078882 non-null  object 
 5   nom_provincia                              1078882 non-null  object 
 6   codi_poblacio                              1078882 non-null  int64  
 7   codi_comarca                               1078867 non-null  float64
 8   codi_provincia                             1078882 non-null  int64  
 9   year                                       1078882 non-null  int64  

Anem a fer la mitjana entre tots els edificis, agrupats per codi districte (mundissec).

In [95]:
df_codi_districte_agg = df_main.groupby('codi_districte').agg({
    # 'referencia_cadastral': 'count',  # Contar cuántos registros hay por distrito
    'codi_postal': 'first',  # Mantener el primer código postal (puede cambiar según necesidad)
    'poblacio': 'first',  # Mantener una muestra de la población
    'comarca': 'first',  # Mantener el nombre de la comarca
    'nom_provincia': 'first',  # Mantener el nombre de la provincia
    'codi_poblacio': 'first',  # Mantener un solo código de población
    'codi_comarca': 'first',  # Mantener un solo código de comarca
    'codi_provincia': 'first',  # Mantener un solo código de provincia
    # 'year': 'mean',  # Promedio del año (puede no tener sentido, revisar)
    'qualificacio_consum_energia_no_renovables': 'mean',  # Moda de la calificación energética
    'renta_neta_mitja_por_persona': 'mean',  # Promedio de la renta neta
    'indice_gini': 'mean',  # Promedio del índice de desigualdad
}).reset_index()

In [96]:
df_codi_districte_agg

,codi_districte,codi_postal,poblacio,comarca,nom_provincia,codi_poblacio,codi_comarca,codi_provincia,qualificacio_consum_energia_no_renovables,renta_neta_mitja_por_persona,indice_gini
0,0800101001,8630,Abrera,Baix Llobregat,Barcelona,8001,11.0,8,5.475524,13629.070922,25.729787
1,0800101002,8630,Abrera,Baix Llobregat,Barcelona,8001,11.0,8,5.598930,12734.123656,26.997312
2,0800101003,8630,Abrera,Baix Llobregat,Barcelona,8001,11.0,8,4.151163,16634.566265,25.174699
3,0800101004,8630,Abrera,Baix Llobregat,Barcelona,8001,11.0,8,5.243902,14973.321608,25.126633
4,0800101005,8630,Abrera,Baix Llobregat,Barcelona,8001,11.0,8,5.262411,12824.834532,23.249640
...,...,...,...,...,...,...,...,...,...,...,...
5080,4390601002,43895,Ampolla (L'),Baix Ebre,Tarragona,43906,9.0,43,5.564286,12132.364734,34.113285
5081,4390701001,43110,Canonja (La),Tarragonès,Tarragona,43907,36.0,43,5.517647,12987.506173,25.944444
5082,4390701002,43110,Canonja (La),Tarragonès,Tarragona,43907,36.0,43,5.570000,11359.744898,26.782653
5083,4390701003,43110,Canonja (La),Tarragonès,Tarragona,43907,36.0,43,5.414286,12613.875912,26.040876


## Calcular ratio

In [60]:
df_costos = pd.read_csv('../../../dades/raposos/costos-projectes-tractats.csv')

In [61]:
df_costos

,Unnamed: 0,provincia,expedients_atorgats,num_habitatges,cost_total_projectes,costos_subvencionats,ajuts_atorgats,mitjana_ajut_m2,ptg_estalvi_energia_final,cost_mitja_rehabilitacio,cost_mitja_subvencionats,cost_mitja_ajuts_atorgats,retorn_inversio
0,0,Barcelona,185,343,46268066.83,36031079.71,13862550.01,88.05,58.32,134892.323120,105046.879621,40415.597697,0.043234
1,1,Girona,58,55,5776568.28,4011626.65,1617672.41,64.27,55.85,105028.514182,72938.666364,29412.225636,0.053176
2,2,Lleida,34,104,4527747.53,2883059.47,1227482.36,78.88,58.20,43536.033942,27721.725673,11802.715000,0.133682
3,3,Tarragona,31,51,2171801.33,1353267.76,601513.36,99.62,66.97,42584.339804,26534.661961,11794.379608,0.157264


In [97]:
df_fusionat = df_codi_districte_agg.merge(df_costos, how='left', left_on='nom_provincia', right_on='provincia')

In [98]:
# Netegem
df_fusionat['qualificacio_consum_energia_no_renovables_norm'] = 1 + ((df_fusionat['qualificacio_consum_energia_no_renovables'] - 1) / 6)
df_fusionat['renta_neta_mitja_por_persona_norm'] = 1 + (df_fusionat['renta_neta_mitja_por_persona'] - df_fusionat['renta_neta_mitja_por_persona'].min()) / (df_fusionat['renta_neta_mitja_por_persona'].max() - df_fusionat['renta_neta_mitja_por_persona'].min())
df_fusionat['indice_gini_norm'] = 1 + (df_fusionat['indice_gini']/100)

In [105]:
# a tener en compte: roi, qualificacio_consum_energia_no_renovables, renta_neta_mitja_por_persona, indice_gini
df_fusionat['ratio'] = (df_fusionat['retorn_inversio']*df_fusionat['qualificacio_consum_energia_no_renovables_norm']*df_fusionat['indice_gini_norm'])/df_fusionat['renta_neta_mitja_por_persona_norm']
df_fusionat['ratio_norm'] = (df_fusionat['ratio'] - df_fusionat['ratio'].min()) / (df_fusionat['ratio'].max() - df_fusionat['ratio'].min())

In [106]:
df_fusionat

,codi_districte,codi_postal,poblacio,comarca,nom_provincia,codi_poblacio,codi_comarca,codi_provincia,qualificacio_consum_energia_no_renovables,renta_neta_mitja_por_persona,...,ptg_estalvi_energia_final,cost_mitja_rehabilitacio,cost_mitja_subvencionats,cost_mitja_ajuts_atorgats,retorn_inversio,qualificacio_consum_energia_no_renovables_norm,renta_neta_mitja_por_persona_norm,indice_gini_norm,ratio,ratio_norm
0,0800101001,8630,Abrera,Baix Llobregat,Barcelona,8001,11.0,8,5.475524,13629.070922,...,58.32,134892.323120,105046.879621,40415.597697,0.043234,1.745921,1.345245,1.257298,0.070549,0.095319
1,0800101002,8630,Abrera,Baix Llobregat,Barcelona,8001,11.0,8,5.598930,12734.123656,...,58.32,134892.323120,105046.879621,40415.597697,0.043234,1.766488,1.312263,1.269973,0.073912,0.105862
2,0800101003,8630,Abrera,Baix Llobregat,Barcelona,8001,11.0,8,4.151163,16634.566265,...,58.32,134892.323120,105046.879621,40415.597697,0.043234,1.525194,1.456009,1.251747,0.056690,0.051870
3,0800101004,8630,Abrera,Baix Llobregat,Barcelona,8001,11.0,8,5.243902,14973.321608,...,58.32,134892.323120,105046.879621,40415.597697,0.043234,1.707317,1.394786,1.251266,0.066220,0.081746
4,0800101005,8630,Abrera,Baix Llobregat,Barcelona,8001,11.0,8,5.262411,12824.834532,...,58.32,134892.323120,105046.879621,40415.597697,0.043234,1.710402,1.315606,1.232496,0.069277,0.091331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5080,4390601002,43895,Ampolla (L'),Baix Ebre,Tarragona,43906,9.0,43,5.564286,12132.364734,...,66.97,42584.339804,26534.661961,11794.379608,0.157264,1.760714,1.290086,1.341133,0.287854,0.776599
5081,4390701001,43110,Canonja (La),Tarragonès,Tarragona,43907,36.0,43,5.517647,12987.506173,...,66.97,42584.339804,26534.661961,11794.379608,0.157264,1.752941,1.321601,1.259444,0.262710,0.697769
5082,4390701002,43110,Canonja (La),Tarragonès,Tarragona,43907,36.0,43,5.570000,11359.744898,...,66.97,42584.339804,26534.661961,11794.379608,0.157264,1.761667,1.261612,1.267827,0.278412,0.746997
5083,4390701003,43110,Canonja (La),Tarragonès,Tarragona,43907,36.0,43,5.414286,12613.875912,...,66.97,42584.339804,26534.661961,11794.379608,0.157264,1.735714,1.307832,1.260409,0.263068,0.698892


In [126]:
df_final = df_fusionat[['codi_districte', 'codi_postal', 'ratio_norm', 'poblacio', 'indice_gini', 'renta_neta_mitja_por_persona', 'qualificacio_consum_energia_no_renovables']].reset_index()

In [129]:
df_final['codi_postal'] = df_final['codi_postal'].apply(lambda x: x.zfill(5))

In [130]:
df_final

,index,codi_districte,codi_postal,ratio_norm,poblacio,indice_gini,renta_neta_mitja_por_persona,qualificacio_consum_energia_no_renovables
0,0,0800101001,08630,0.095319,Abrera,25.729787,13629.070922,5.475524
1,1,0800101002,08630,0.105862,Abrera,26.997312,12734.123656,5.598930
2,2,0800101003,08630,0.051870,Abrera,25.174699,16634.566265,4.151163
3,3,0800101004,08630,0.081746,Abrera,25.126633,14973.321608,5.243902
4,4,0800101005,08630,0.091331,Abrera,23.249640,12824.834532,5.262411
...,...,...,...,...,...,...,...,...
5080,5080,4390601002,43895,0.776599,Ampolla (L'),34.113285,12132.364734,5.564286
5081,5081,4390701001,43110,0.697769,Canonja (La),25.944444,12987.506173,5.517647
5082,5082,4390701002,43110,0.746997,Canonja (La),26.782653,11359.744898,5.570000
5083,5083,4390701003,43110,0.698892,Canonja (La),26.040876,12613.875912,5.414286


In [131]:
df_final['ratio_norm'].min()

0.0

In [132]:
df_final['ratio_norm'].max()

1.0

In [133]:
df_final.to_csv('../../../dades/raposos/mapa_socioeconomic.csv')